In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# imports for training

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


ModuleNotFoundError: No module named 'darts'

In [2]:
# from benchmarks.mqrnn import MQRNN
# import tensorflow as tf
import pandas as pd
# import torch
# import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'calls911_benchmarks'
data = pd.read_csv('./datasets/text_data/calls911/'+dataset_name+'.csv')
# data["time_idx"] = data.index.to_list()
# data_melted = pd.melt(data, id_vars=['date','time_idx'])
# data_melted

,date,time_idx,variable,value
0,2015-12-01,0,ABINGTON,514.0
1,2016-01-01,1,ABINGTON,727.0
2,2016-02-01,2,ABINGTON,713.0
3,2016-03-01,3,ABINGTON,668.0
4,2016-04-01,4,ABINGTON,728.0
...,...,...,...,...
3467,2020-03-01,51,WORCESTER,56.0
3468,2020-04-01,52,WORCESTER,55.0
3469,2020-05-01,53,WORCESTER,83.0
3470,2020-06-01,54,WORCESTER,82.0


In [3]:
treated = ["ABINGTON",  "AMBLER",  "CHELTENHAM",  "COLLEGEVILLE",  "CONSHOHOCKEN", 
                   "EAST GREENVILLE",  "EAST NORRITON",  "FRANCONIA" , "GREEN LANE", "HATFIELD TOWNSHIP", 
                   "HORSHAM" , "JENKINTOWN",  "LANSDALE",  "LIMERICK",  "LOWER GWYNEDD", 
                   "LOWER MERION",  "LOWER MORELAND",  "LOWER POTTSGROVE",  "LOWER PROVIDENCE",  "LOWER SALFORD", 
                   "MARLBOROUGH",  "MONTGOMERY",  "NARBERTH",  "PENNSBURG",  "PERKIOMEN", 
                   "PLYMOUTH",  "POTTSTOWN",  "RED HILL",  "ROCKLEDGE",  "ROYERSFORD", 
                   "SCHWENKSVILLE",  "SKIPPACK",  "SOUDERTON",  "TELFORD",  "TOWAMENCIN", 
                   "UPPER DUBLIN",  "UPPER FREDERICK",  "UPPER GWYNEDD",  "UPPER HANOVER",  "UPPER MERION", 
                   "UPPER MORELAND",  "UPPER POTTSGROVE",  "UPPER PROVIDENCE",  "UPPER SALFORD",  "WEST CONSHOHOCKEN", 
                   "WEST NORRITON",  "WEST POTTSGROVE",  "WHITEMARSH",  "WHITPAIN",  "WORCESTER"]
control = ["BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO", 
                   "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD", 
                   "SPRINGFIELD", "TRAPPE"]
                   

In [8]:
# Create a binary mask indicating whether each column is treated (1) or control (0)
data_melted['tnc'] = [1 if col in treated else 0 for col in data_melted.variable]
data_melted

,date,time_idx,variable,value,tnc
0,2015-12-01,0,ABINGTON,514.0,1
1,2016-01-01,1,ABINGTON,727.0,1
2,2016-02-01,2,ABINGTON,713.0,1
3,2016-03-01,3,ABINGTON,668.0,1
4,2016-04-01,4,ABINGTON,728.0,1
...,...,...,...,...,...
3467,2020-03-01,51,WORCESTER,56.0,1
3468,2020-04-01,52,WORCESTER,55.0,1
3469,2020-05-01,53,WORCESTER,83.0,1
3470,2020-06-01,54,WORCESTER,82.0,1


In [52]:
from preprocess_scripts.data_loader import DataLoader
dataset_name = 'calls911_benchmarks'
batch_size = 21
input_size = 56
forecast_horizon=7
feature_type='MS'
target='ABINGTON'
data_loader = DataLoader(dataset_name,batch_size,input_size,forecast_horizon,feature_type, target)

42 7 7


In [69]:
all_columns = data.columns.tolist()

# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a PyTorch tensor
cate_tensor = torch.tensor(cate_mask, dtype=torch.float32).view(1, 1, -1)

NameError: name 'torch' is not defined

In [70]:
cate_mask

[0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [53]:
train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

In [64]:
for batch in train_data:
    x,y = batch

In [67]:
batch[0]

<tf.Tensor: shape=(21, 15, 62), dtype=float32, numpy=
array([[[-1.63818628e-01, -1.74985635e+00, -5.34819961e-01, ...,
         -1.09709918e+00,  9.75768030e-01, -1.79793131e+00],
        [-7.55820930e-01, -1.25494754e+00, -6.17402494e-01, ...,
         -1.12326503e+00, -1.01767220e-01, -1.36080766e+00],
        [ 3.35154720e-02, -4.30099368e-01, -1.11289752e+00, ...,
         -1.80357873e+00, -2.99315359e-02,  5.20385336e-03],
        ...,
        [-1.00577748e+00, -1.00749302e+00, -1.85613990e+00, ...,
         -1.43725598e+00, -2.36459136e+00, -2.13357985e-01],
        [-1.50663018e-01, -1.25494754e+00, -1.60839248e+00, ...,
         -1.04476738e+00,  2.21493363e-01,  8.79451215e-01],
        [ 3.49250019e-01,  1.47294313e-01, -5.34819961e-01, ...,
         -1.09709918e+00, -1.75398791e+00, -9.78324413e-01]],

       [[-8.48849863e-02,  1.38456655e+00,  4.56169993e-01, ...,
         -7.04610527e-01,  4.01082575e-01, -1.04077064e-01],
        [ 2.70316392e-01,  6.48094937e-02,  2.084

In [68]:
batch[1]

<tf.Tensor: shape=(21, 7, 1), dtype=float32, numpy=
array([[[-0.08488499],
        [ 0.2703164 ],
        [-0.09804059],
        [ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124]],

       [[-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215]],

       [[ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215],
        [-0.2953747 ],
        [-0.07172938],
        [ 0.7702294 ]],

       [[-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215],
        [-0.2953747 ]],

       [[-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ]],

       [[-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.571642

In [55]:
val_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 56, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [56]:
test_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 56, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [16]:
# import pandas as pd
# pd.read_csv('./datasets/text_data/calls911/weather.csv').dtypes

In [30]:
from benchmarks.mqrnn import MQRNN
import tensorflow as tf
import pandas as pd
# import torch
# import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'calls911_benchmarks'
data = pd.read_csv('./datasets/text_data/calls911/'+dataset_name+'.csv')
# Define parameters for MQRNN model
d_model = data_loader.n_feature  # Adjust based on your data
# d_model = 62  # You can adjust this value
tau = 7  # You can adjust this value
num_targets = 1  # You can adjust this value
num_quantiles = 3  # You can adjust this value based on your task
n_layers = 2  # You can adjust this value
dr = 0.1  # You can adjust this value
num_epochs = 5
learning_rate = 0.0001

# Update the MQRNN model instantiation
mqrnn_model = MQRNN(
    d_model=d_model,
    tau=tau,
    num_targets=num_targets,
    num_quantiles=num_quantiles,
    n_layers=n_layers,
    dr=dr
)

# Loss function and optimizer
criterion = tf.keras.losses.MeanSquaredError()  # Replace with your actual loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

all_columns = data.columns.tolist()[1:]

# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a TensorFlow tensor
cate_tensor = tf.constant([cate_mask], dtype=tf.float32)

for epoch in range(num_epochs):
    for batch in train_data:
        conti, forecast_tensor = batch

        # # Create cate_tensor based on your criteria
        # treated_unit_index = 0
        # future_tensor = tf.zeros_like(forecast_tensor)
        # future_tensor[:, :, treated_unit_index] = 1

        # # Concatenate conti, cate_tensor, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor, future_tensor], axis=-1)
        # Create cate_tensor based on your criteria
        treated_unit_index = 0
        future_tensor = tf.zeros_like(forecast_tensor)
        future_tensor = tf.tensor_scatter_nd_add(future_tensor, indices=[(0, 0, treated_unit_index)], updates=[1.0])

        # Assuming cate_tensor has shape [1, 1, 63]
        # Expand dimensions of cate_tensor to match conti and future_tensor
        cate_tensor_expanded = tf.expand_dims(cate_tensor, axis=0)  # Assuming axis=0 is the batch dimension

        # # Repeat the expanded cate_tensor to match the batch size of conti and future_tensor
        cate_tensor_expanded = tf.repeat(cate_tensor_expanded, repeats=tf.shape(conti)[0], axis=0)

        # # Now, cate_tensor_expanded has shape [5, 1, 62], matching the other tensors
        # # Concatenate conti, cate_tensor_expanded, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor_expanded, future_tensor], axis=-1)
        # # Now, cate_tensor_expanded has shape [1, 1, 63], matching the other tensors
        # Concatenate conti, cate_tensor_expanded, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor, future_tensor], axis=-1)

        with tf.GradientTape() as tape:
            print('here')
            output = mqrnn_model.call(conti, cate_tensor_expanded, future_tensor)

            # Reshape the target tensor to match the output
            target = forecast_tensor

            loss = criterion(target, output)

        gradients = tape.gradient(loss, mqrnn_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, mqrnn_model.trainable_variables))

here


ValueError: Exception encountered when calling layer 'global_decoder_8' (type GlobalDecoder).

not enough values to unpack (expected 3, got 2)

Call arguments received by layer 'global_decoder_8' (type GlobalDecoder):
  • future=tf.Tensor(shape=(21, 7, 1), dtype=float32)
  • hidden=tf.Tensor(shape=(21, 62), dtype=float32)

In [28]:
train_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 15, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [39]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model, n_layers, dr):
        super(Encoder, self).__init__()
        # Define your model
        self.lstm = tf.keras.Sequential()
        for _ in range(n_layers):
            self.lstm.add(tf.keras.layers.LSTM(units=d_model, return_state=True, dropout=dr))

    def call(self, conti, cate):
        # Use cate in the encoder
        x = tf.concat([conti, cate], axis=1)

        _, hidden, cell = self.lstm(x)
        # _, hidden, cell = self.lstm(inputs)
        return hidden, cell

encoder = Encoder(d_model, n_layers, dr)
hidden, _ = encoder(conti, cate_tensor_expanded)

ValueError: Exception encountered when calling layer 'sequential_7' (type Sequential).

All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

Call arguments received by layer 'sequential_7' (type Sequential):
  • inputs=tf.Tensor(shape=(21, 16, 62), dtype=float32)
  • training=None
  • mask=None

In [40]:
lstm = tf.keras.Sequential()
for _ in range(2):
    lstm.add(tf.keras.layers.LSTM(units=d_model, return_state=True, dropout=dr))


In [43]:
x = tf.concat([conti, cate_tensor_expanded], axis=1)
x

<tf.Tensor: shape=(21, 16, 62), dtype=float32, numpy=
array([[[ 4.0187246e-01, -6.7755383e-01, -8.6514997e-01, ...,
         -5.4761511e-01,  1.4965768e-01, -5.9584123e-01],
        [-1.0057775e+00, -1.0074930e+00, -1.8561399e+00, ...,
         -1.4372560e+00, -2.3645914e+00, -2.1335799e-01],
        [-1.5066302e-01, -1.2549475e+00, -1.6083925e+00, ...,
         -1.0447674e+00,  2.2149336e-01,  8.7945122e-01],
        ...,
        [-7.9528773e-01, -1.0016013e-01,  1.2583999e-01, ...,
         -8.0927420e-01, -1.4307274e+00,  8.2481074e-01],
        [ 4.5458884e+00,  2.2094147e+00,  7.0391750e-01, ...,
          2.6446257e+00,  2.5202353e+00,  2.4093840e+00],
        [ 1.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00]],

       [[-2.7817843e+00, -2.4097350e+00, -1.9387225e+00, ...,
         -2.7978833e+00, -2.2209198e+00, -2.5082574e+00],
        [ 2.0359866e-02,  3.1226394e-01,  3.7358749e-01, ...,
         -1.0279464e-01,  5.0

In [14]:
hidden

<tf.Tensor: shape=(15, 62), dtype=float32, numpy=
array([[-7.29743987e-02, -1.92274880e-02, -8.56410787e-02,
        -1.55965686e-01,  5.65690920e-02, -1.55455852e-02,
         4.61784936e-02,  2.18769088e-01,  5.16066067e-02,
         9.83765870e-02, -6.47029206e-02,  1.89189047e-01,
         2.85775393e-01, -4.04633015e-01,  2.85761565e-01,
        -6.93058372e-02,  3.49319214e-03,  1.46318585e-01,
         2.90825278e-01,  3.67854446e-01,  3.28221411e-01,
        -3.01147729e-01, -3.24748993e-01, -3.29498649e-01,
         2.05114797e-01,  1.12396898e-02, -3.97291966e-02,
         2.17170507e-01, -4.28277962e-02, -3.20963413e-01,
        -1.75980344e-01, -7.87312239e-02, -3.56796607e-02,
        -2.12128937e-01, -7.44436681e-02, -4.83417481e-01,
        -2.66829953e-02, -8.01192448e-02, -2.05800727e-01,
        -5.37466943e-01, -1.10658482e-01, -1.17170000e-02,
        -1.64912179e-01,  2.75728136e-01,  3.81623693e-02,
         2.00527851e-02, -4.80817735e-01, -4.11651433e-01,
      

In [10]:
tf.expand_dims(future_tensor, axis=-1)

<tf.Tensor: shape=(5, 7, 1, 1), dtype=float32, numpy=
array([[[[1.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]]], dtype=float32)>

In [9]:
tf.expand_dims(hidden, axis=-1)  # Assuming axis=0 is the batch dimension


<tf.Tensor: shape=(5, 62, 1), dtype=float32, numpy=
array([[[-0.00316266],
        [-0.00064736],
        [-0.19473381],
        [ 0.03555915],
        [-0.02805869],
        [-0.09891547],
        [ 0.03250901],
        [-0.1499267 ],
        [ 0.03983374],
        [-0.1209138 ],
        [ 0.09328417],
        [-0.05761017],
        [-0.40352422],
        [-0.04340634],
        [ 0.31057784],
        [ 0.06992783],
        [-0.18012914],
        [-0.09889585],
        [ 0.13981047],
        [ 0.04054002],
        [ 0.04222865],
        [-0.11550292],
        [-0.06337526],
        [ 0.11566366],
        [-0.21819176],
        [-0.05861421],
        [ 0.18389235],
        [ 0.2039906 ],
        [ 0.19449301],
        [ 0.19363399],
        [ 0.09303418],
        [ 0.25356674],
        [ 0.03670028],
        [ 0.06142342],
        [ 0.21268721],
        [ 0.04982827],
        [-0.08481552],
        [-0.12171865],
        [-0.18368185],
        [ 0.06950779],
        [ 0.10259786],
     

In [5]:
conti

<tf.Tensor: shape=(5, 15, 62), dtype=float32, numpy=
array([[[-0.75582093, -1.2549475 , -0.6174025 , ..., -1.123265  ,
         -0.10176722, -1.3608077 ],
        [ 0.03351547, -0.43009937, -1.1128975 , ..., -1.8035787 ,
         -0.02993154,  0.00520385],
        [-0.84791017, -0.595069  ,  1.6949074 , ..., -0.5999469 ,
          0.36516473, -0.9783244 ],
        ...,
        [-0.15066302, -1.2549475 , -1.6083925 , ..., -1.0447674 ,
          0.22149336,  0.8794512 ],
        [ 0.34925002,  0.14729431, -0.53481996, ..., -1.0970992 ,
         -1.7539879 , -0.9783244 ],
        [-0.08488499,  1.3845665 ,  0.45617   , ..., -0.7046105 ,
          0.40108258, -0.10407706]],

       [[ 0.40187246, -0.67755383, -0.86515   , ..., -0.5476151 ,
          0.14965768, -0.5958412 ],
        [-1.0057775 , -1.007493  , -1.8561399 , ..., -1.437256  ,
         -2.3645914 , -0.21335799],
        [-0.15066302, -1.2549475 , -1.6083925 , ..., -1.0447674 ,
          0.22149336,  0.8794512 ],
        ...,
 

In [1]:
from benchmarks.mqrnn import MQRNN
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from preprocess_scripts.data_loader import DataLoader
dataset_name = 'calls911_benchmarks'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = pd.read_csv('./datasets/text_data/calls911/' + dataset_name + '.csv')
batch_size = 5
input_size = 15
forecast_horizon=7
feature_type='MS'
target='ABINGTON'
data_loader = DataLoader(dataset_name,batch_size,input_size,forecast_horizon,feature_type, target)

# Define parameters for MQRNN model
d_input = data_loader.n_feature  # Adjust based on your data
d_model = 62  # You can adjust this value
tau = 7  # You can adjust this value
num_targets = 1  # You can adjust this value
num_quantiles = 3  # You can adjust this value based on your task
n_layers = 2  # You can adjust this value
dr = 0.1  # You can adjust this value
num_epochs = 5
learning_rate = 0.0001

# Update the MQRNN model instantiation
mqrnn_model = MQRNN(
    d_input=d_input,
    d_model=d_model,
    tau=tau,
    num_targets=num_targets,
    num_quantiles=num_quantiles,
    n_layers=n_layers,
    dr=dr
).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Replace with your actual loss function
optimizer = optim.Adam(mqrnn_model.parameters(), lr=learning_rate)

train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

all_columns = data.columns.tolist()


# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a PyTorch tensor
cate_tensor = torch.tensor(cate_mask, dtype=torch.float32).view(1, 1, -1).to(device)

for epoch in range(num_epochs):
    for batch in train_data:
        conti, forecast_tensor = batch

        # Create cate_tensor based on your criteria
        treated_unit_index = 0
        future_tensor = torch.zeros_like(forecast_tensor).to(device)
        future_tensor[:, :, treated_unit_index] = 1

        # Concatenate conti, cate_tensor, and future_tensor along the last dimension
        input_tensor = torch.cat([conti, cate_tensor, future_tensor], dim=-1).to(device)

        optimizer.zero_grad()
        output = mqrnn_model(input_tensor)

        # Reshape the target tensor to match the output
        target = forecast_tensor.to(device)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

2024-01-27 17:53:25.107128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 17:53:25.107204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 17:53:25.108971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 17:53:27.236343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


42 22 22


NameError: name 'treated' is not defined

TypeError: zeros_like(): argument 'input' (position 1) must be Tensor, not tensorflow.python.framework.ops.EagerTensor

In [28]:
conti

<tf.Tensor: shape=(1, 15, 62), dtype=float32, numpy=
array([[[ 3.35154720e-02, -4.30099368e-01, -1.11289752e+00,
         -1.21783900e+00, -3.15799624e-01, -1.53396741e-01,
         -5.50933659e-01,  8.85367215e-01,  8.16210389e-01,
         -1.52368045e+00, -2.90683061e-01, -9.78549778e-01,
         -8.48706186e-01,  1.12148929e+00, -6.56743348e-01,
         -5.20974219e-01, -8.50343406e-01,  1.08714633e-01,
         -4.56157058e-01, -1.27903569e+00, -7.68515944e-01,
         -1.91132426e-01,  2.69135892e-01, -7.37774611e-01,
         -7.84782469e-01,  3.05532515e-01, -1.48782730e-01,
         -1.17373967e+00, -1.07498932e+00, -8.86303723e-01,
          9.65268135e-01, -1.33656108e+00, -1.51987255e+00,
         -9.55359519e-01, -5.83113497e-03, -8.71230781e-01,
          1.56340134e+00, -6.12545907e-01,  9.10261750e-01,
         -3.97071630e-01, -1.02930643e-01, -3.79987121e-01,
          1.74299642e-01, -1.23852587e+00, -1.72188854e+00,
         -5.07075906e-01, -2.52038896e-01, -1.2

In [29]:
future

<tf.Tensor: shape=(1, 7, 1), dtype=float32, numpy=
array([[[-0.09804059],
        [ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486]]], dtype=float32)>

In [11]:
import torch
tmp_feature_list = []
emb_output = torch.cat(tmp_feature_list, axis=-2)
emb_output = emb_output.view(conti.size(0), conti.size(1), -1)

x = torch.cat([conti, emb_output], axis=-1)

x


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [10]:
cate

<tf.Tensor: shape=(5, 7, 1), dtype=float32, numpy=
array([[[ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486],
        [-0.7952877 ],
        [ 4.5458884 ],
        [-0.5716424 ]],

       [[-0.5716424 ],
        [ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626]],

       [[ 4.5458884 ],
        [-0.5716424 ],
        [ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ]],

       [[ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ]],

       [[ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486],
        [-0.7952877 ]]], dtype=float32)>

In [1]:
# !pip install pycausalimpact
from causalimpact import CausalImpact

In [23]:
import pandas as pd
# import torch
# import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'calls911_benchmarks'
data_row = pd.read_csv('./datasets/text_data/calls911/'+dataset_name+'.csv')

In [18]:
forecast_horizon=7

In [25]:
data_row

,date,ABINGTON,AMBLER,BRIDGEPORT,BRYN ATHYN,CHELTENHAM,COLLEGEVILLE,CONSHOHOCKEN,DOUGLASS,EAST GREENVILLE,...,UPPER MORELAND,UPPER POTTSGROVE,UPPER PROVIDENCE,UPPER SALFORD,WEST CONSHOHOCKEN,WEST NORRITON,WEST POTTSGROVE,WHITEMARSH,WHITPAIN,WORCESTER
0,2015-12-01,514.0,49.0,43.0,19.0,339.0,43.0,51.0,63.0,18.0,...,276.0,46.0,191.0,11.0,81.0,121.0,32.0,213.0,182.0,64.0
1,2016-01-01,727.0,82.0,71.0,15.0,586.0,58.0,110.0,146.0,31.0,...,460.0,99.0,285.0,34.0,107.0,202.0,75.0,316.0,258.0,95.0
2,2016-02-01,713.0,57.0,60.0,18.0,548.0,89.0,95.0,109.0,27.0,...,389.0,65.0,281.0,42.0,79.0,182.0,83.0,278.0,271.0,77.0
3,2016-03-01,668.0,63.0,59.0,18.0,488.0,52.0,111.0,121.0,13.0,...,338.0,52.0,248.0,19.0,76.0,168.0,52.0,277.0,241.0,85.0
4,2016-04-01,728.0,73.0,53.0,16.0,531.0,52.0,94.0,115.0,29.0,...,372.0,69.0,256.0,33.0,82.0,182.0,48.0,251.0,243.0,110.0
5,2016-05-01,661.0,71.0,87.0,24.0,511.0,56.0,86.0,82.0,13.0,...,428.0,49.0,245.0,37.0,80.0,202.0,70.0,297.0,254.0,92.0
6,2016-06-01,674.0,81.0,66.0,26.0,560.0,31.0,91.0,105.0,22.0,...,425.0,57.0,279.0,27.0,96.0,199.0,39.0,332.0,270.0,94.0
7,2016-07-01,721.0,74.0,55.0,26.0,559.0,50.0,127.0,116.0,34.0,...,428.0,62.0,263.0,17.0,75.0,194.0,59.0,322.0,225.0,96.0
8,2016-08-01,751.0,56.0,74.0,14.0,572.0,57.0,114.0,95.0,28.0,...,409.0,53.0,304.0,46.0,97.0,204.0,73.0,315.0,227.0,76.0
9,2016-09-01,693.0,71.0,59.0,25.0,557.0,80.0,100.0,110.0,23.0,...,407.0,57.0,311.0,31.0,106.0,190.0,54.0,273.0,247.0,115.0


In [34]:
y_pred_list = []
for i in data_row.columns[1:]:
   ci = CausalImpact(data_row.loc[:,[i] + [col for col in \
            data_row.columns[1:] if col != i]],
         [0,len(data_row['date'])-forecast_horizon-1],
         [len(data_row['date'])-forecast_horizon,
         len(data_row['date'])-1])
   # evaluate the model
   y_pred = ci.inferences.loc[(len(data_row['date'])-\
            forecast_horizon):(len(data_row['date'])-1),'preds']
   y_pred_list.append(y_pred)

/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: standardize, nseasons. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score.

In [37]:
dataset_name = 'calls911_benchmarks'


In [39]:
output = 'results/benchmarks/predicted/' + dataset_name + '_causalimpact.txt'

In [2]:
import numpy as np

In [3]:
y_pred_df= pd.read_csv("results/nn_model_results/rnn/processed_ensemble_forecasts/sim_10_60_l_he_LSTMcell_cocob_without_stl_decomposition_0.5.txt", header=None)
y_pred_df

NameError: name 'pd' is not defined

In [44]:
num_time_series = len(y_pred_df.index) 
num_time_series

10

In [1]:
dataset_type = "sim"
dataset_name = "sim_10_60_l_he"

In [47]:
y_true_df_A = pd.read_csv("datasets/text_data/" + dataset_type +  \
    "/" + dataset_name + "_test_actual.csv")
# Reading the original data to calculate the MASE errors
y_true_df_B = pd.read_csv("datasets/text_data/" + dataset_type +  \
    "/" + dataset_name + "_train.csv")
data_row_A = y_true_df_A.pivot(index='time', columns='series_id', values='value')
data_row_B = y_true_df_B.pivot(index='time', columns='series_id', values='value')
data_row = pd.concat([data_row_B, data_row_A],ignore_index=True)
data_row_A = data_row_A.T
data_row_B = data_row_B.T

In [51]:
y_pred_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.629724,6.708806,6.573927,6.606805,6.469340,6.340422,6.221756,6.121868,5.726090,5.626291,5.356708,5.415047
1,6.310534,6.200516,6.156707,6.083326,6.093747,6.103312,6.005089,6.035113,6.110431,6.200161,6.303062,6.606723
2,1.688976,1.691827,1.816579,1.901221,2.242351,2.412698,2.542423,2.654905,2.768438,2.768851,2.828596,2.972987
3,4.135915,4.376139,4.423117,4.481943,4.158025,3.853407,3.646660,3.328388,2.983174,3.024314,2.793119,2.867337
4,2.734496,2.946369,3.119134,3.080817,2.595675,2.472294,2.780016,3.059133,3.283427,2.750075,1.842087,1.239818
5,9.942944,9.710226,9.403375,9.216754,8.964634,8.856621,8.872820,8.870498,8.984103,9.164515,9.741502,9.684794
6,3.030451,3.039801,3.200372,3.355505,3.887331,4.240273,4.410506,4.705541,4.621328,4.357219,4.171184,4.403385
7,12.576810,12.840830,12.976706,13.129715,12.596846,12.314832,12.065861,11.859237,11.461786,11.189771,10.925467,10.848368
8,10.442896,10.540597,10.592790,10.791485,10.696101,10.628230,10.654056,10.762355,10.436309,10.143882,9.852842,9.695397
9,6.132347,6.080677,6.031597,6.025407,6.196535,6.324553,6.439402,6.533186,6.632051,6.616722,6.757506,6.674554


In [52]:
y_true_df_B

,series_id,time,value,c_t,time_series_length,amount_of_time_series,dgp,te_intervention
0,10_1_60_linear_heterogeneous,1,4.000000,control,60,10,linear,heterogeneous
1,10_2_60_linear_heterogeneous,1,4.000000,control,60,10,linear,heterogeneous
2,10_3_60_linear_heterogeneous,1,4.000000,control,60,10,linear,heterogeneous
3,10_4_60_linear_heterogeneous,1,4.000000,control,60,10,linear,heterogeneous
4,10_5_60_linear_heterogeneous,1,4.000000,control,60,10,linear,heterogeneous
...,...,...,...,...,...,...,...,...
475,10_6_60_linear_heterogeneous,48,9.990315,treated,60,10,linear,heterogeneous
476,10_7_60_linear_heterogeneous,48,3.043457,treated,60,10,linear,heterogeneous
477,10_8_60_linear_heterogeneous,48,12.360816,treated,60,10,linear,heterogeneous
478,10_9_60_linear_heterogeneous,48,10.435537,treated,60,10,linear,heterogeneous


In [50]:
data_row_A

time,49,50,51,52,53,54,55,56,57,58,59,60
series_id,,,,,,,,,,,,
10_10_60_linear_heterogeneous,6.345929,6.504824,6.825071,6.582064,6.828245,6.541944,6.735053,6.955601,6.737537,6.496371,6.283894,5.751389
10_1_60_linear_heterogeneous,6.378587,6.152957,5.852596,6.090985,6.367565,6.807419,7.006682,7.163756,6.792528,6.509721,6.274091,5.958255
10_2_60_linear_heterogeneous,6.418603,6.869903,7.254626,7.166583,7.134166,6.563448,6.348515,6.497834,6.242976,5.796329,5.375151,4.985877
10_3_60_linear_heterogeneous,1.326354,1.456909,1.594475,2.159289,3.119506,3.677154,3.487978,3.143146,2.858583,2.672392,2.159707,1.447951
10_4_60_linear_heterogeneous,4.238426,4.366579,4.770059,4.225180,4.357252,4.460943,4.765281,4.812774,5.467025,5.657111,5.536041,6.042750
10_5_60_linear_heterogeneous,3.305205,3.460611,3.428427,3.619114,3.786007,3.757834,3.931773,4.299621,4.329619,4.414485,4.389320,4.032280
10_6_60_linear_heterogeneous,9.617934,9.426890,9.041199,8.898806,8.518961,8.204192,7.634701,7.409369,7.507990,6.546740,5.857119,5.331190
10_7_60_linear_heterogeneous,2.880725,2.291242,1.940018,1.746893,1.750235,1.905635,2.479341,2.418841,2.676017,2.850685,3.149199,3.879921
10_8_60_linear_heterogeneous,15.099995,15.001862,12.792860,15.069770,15.151043,15.578149,15.005937,12.775508,12.766202,12.455028,12.585050,12.673118


AttributeError: 'Index' object has no attribute 'split'

In [62]:
# Custom sorting function to extract the numeric part
def custom_sort_key(s):
    parts = s.split('_')
    return int(parts[1])

# Use the custom sorting function as the key
data_row_A.loc[sorted(data_row_A.index, key=custom_sort_key),:]


time,49,50,51,52,53,54,55,56,57,58,59,60
series_id,,,,,,,,,,,,
10_1_60_linear_heterogeneous,6.378587,6.152957,5.852596,6.090985,6.367565,6.807419,7.006682,7.163756,6.792528,6.509721,6.274091,5.958255
10_2_60_linear_heterogeneous,6.418603,6.869903,7.254626,7.166583,7.134166,6.563448,6.348515,6.497834,6.242976,5.796329,5.375151,4.985877
10_3_60_linear_heterogeneous,1.326354,1.456909,1.594475,2.159289,3.119506,3.677154,3.487978,3.143146,2.858583,2.672392,2.159707,1.447951
10_4_60_linear_heterogeneous,4.238426,4.366579,4.770059,4.225180,4.357252,4.460943,4.765281,4.812774,5.467025,5.657111,5.536041,6.042750
10_5_60_linear_heterogeneous,3.305205,3.460611,3.428427,3.619114,3.786007,3.757834,3.931773,4.299621,4.329619,4.414485,4.389320,4.032280
10_6_60_linear_heterogeneous,9.617934,9.426890,9.041199,8.898806,8.518961,8.204192,7.634701,7.409369,7.507990,6.546740,5.857119,5.331190
10_7_60_linear_heterogeneous,2.880725,2.291242,1.940018,1.746893,1.750235,1.905635,2.479341,2.418841,2.676017,2.850685,3.149199,3.879921
10_8_60_linear_heterogeneous,15.099995,15.001862,12.792860,15.069770,15.151043,15.578149,15.005937,12.775508,12.766202,12.455028,12.585050,12.673118
10_9_60_linear_heterogeneous,10.254313,10.363391,10.535353,11.079612,11.118020,11.333678,11.225567,10.941201,11.048612,9.987595,9.345481,8.400910


In [49]:
data_row_B

time,1,2,3,4,5,6,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
series_id,,,,,,,,,,,,,,,,,,,,,
10_10_60_linear_heterogeneous,4.0,3.853568,4.233268,4.354999,4.388512,5.057418,5.187652,4.928620,5.018153,5.618299,...,6.898193,7.066208,6.920042,6.698136,7.053589,7.045750,7.179429,6.821644,6.380702,6.305574
10_1_60_linear_heterogeneous,4.0,2.683460,1.788648,1.832475,1.848978,2.591947,3.307043,3.251489,3.391611,3.149968,...,4.521239,5.181117,5.613029,6.903968,7.841402,7.983405,7.823119,7.068117,7.153280,6.751326
10_2_60_linear_heterogeneous,4.0,2.974253,2.948683,3.672360,4.015603,4.512820,4.663110,5.300558,5.740905,6.461263,...,6.841498,6.915100,6.751896,6.746984,6.382577,5.755363,5.446491,5.699152,5.941987,6.121536
10_3_60_linear_heterogeneous,4.0,4.470470,3.694064,2.805391,2.767085,3.000050,3.037031,3.788352,3.730273,3.909317,...,4.996539,4.710407,4.450999,3.638493,2.947795,2.671857,2.543497,2.249676,1.955008,1.670262
10_4_60_linear_heterogeneous,4.0,4.753893,6.042675,5.531434,5.578234,5.160179,4.827438,4.500598,4.671222,4.272978,...,2.288709,2.805814,3.237261,3.708810,3.612122,4.136078,3.892096,4.028035,3.866400,4.075891
10_5_60_linear_heterogeneous,4.0,3.059531,3.446069,3.153067,3.138960,2.827227,2.548592,2.833065,3.293848,3.872409,...,1.037410,0.431445,0.364150,0.333668,1.065808,1.816086,2.167989,2.614635,3.029120,2.841879
10_6_60_linear_heterogeneous,4.0,3.274956,2.927431,3.431322,4.344974,4.314408,4.935989,5.225889,5.136426,5.465214,...,8.678591,8.716804,9.017681,8.956676,8.436440,8.168121,8.801920,9.594553,10.126045,9.990315
10_7_60_linear_heterogeneous,4.0,3.990011,4.110910,4.712161,4.871238,5.454914,6.421490,6.795303,6.207478,5.692347,...,5.982678,5.790201,6.103956,6.568936,6.067351,5.715336,5.164559,4.391121,3.957237,3.043457
10_8_60_linear_heterogeneous,4.0,5.259868,6.583133,7.222388,7.776269,8.587760,8.457479,8.074153,8.020679,7.699433,...,9.297328,9.760329,9.939821,10.532298,11.095576,10.941753,11.136625,11.057943,12.054023,12.360816


In [48]:

# SMAPE
time_series_wise_SMAPE = 2 * np.abs(y_pred_df - np.array(data_row_A)) /\
    (np.abs(y_pred_df) + np.abs(np.array(data_row_A)))
SMAPEPerSeries = np.mean(time_series_wise_SMAPE, axis=1)
mean_SMAPE = np.mean(SMAPEPerSeries)
mean_SMAPE_str = f"mean_SMAPE:{mean_SMAPE}"
print(mean_SMAPE_str)

mase_vector = []
for i in range(num_time_series):
    lagged_diff = [data_row_B.iloc[i,j] - \
                data_row_B.iloc[i,j - forecast_horizon]\
                    for j in range(forecast_horizon,\
                    len(data_row_B.columns))]
    mase_vector.append(np.mean(np.abs(np.array(np.array(data_row_A.iloc[i]))\
                - np.array(y_pred_df.iloc[i])) / np.mean(np.abs(lagged_diff))))

mean_MASE = np.mean(mase_vector)
mean_MASE_str = f"mean_MASE:{mean_MASE}"
print(mean_MASE_str)

mean_SMAPE:0.5624922298939096
mean_MASE:2.839062123749925


In [1]:
############ tsmixer
%load_ext autoreload
%autoreload 2
import benchmarks
import time
import glob
import os
from preprocess_scripts.data_loader import DataLoader
import tensorflow as tf
import numpy as np
import pandas as pd

# first sort, then do placebo test
def custom_sort_key(s):
    parts = s.split('_')
    return int(parts[1])

def sMAPE_tf(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    smape_values = tf.abs(y_pred - y_true) / (tf.abs(y_pred) + tf.abs(y_true)) * 2
    smape_per_series = tf.reduce_mean(smape_values, axis=1)
    mean_smape = tf.reduce_mean(smape_per_series)

    return mean_smape  # Convert TensorFlow tensor to NumPy array for compatibility


2024-02-08 00:20:52.388715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 00:20:52.388873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 00:20:52.391844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 00:20:56.312005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
dataset_name = 'sim_10_60_l_he'

In [8]:
y_true_df_A = pd.read_csv("./datasets/text_data/" + dataset_type +  \
        "/" + dataset_name + "_test_actual.csv")
# Reading the original data to calculate the MASE errors
y_true_df_B = pd.read_csv("./datasets/text_data/" + dataset_type +  \
        "/" + dataset_name + "_train.csv")
data_row_A = y_true_df_A.pivot(index='time', columns='series_id', values='value')
data_row_B = y_true_df_B.pivot(index='time', columns='series_id', values='value')
data_row = pd.concat([data_row_B, data_row_A],ignore_index=True)

In [12]:
data_row.to_csv("./datasets/text_data/sim/"+dataset_name+".csv", index=False)

In [17]:
data_row_A

series_id,10_10_60_linear_heterogeneous,10_1_60_linear_heterogeneous,10_2_60_linear_heterogeneous,10_3_60_linear_heterogeneous,10_4_60_linear_heterogeneous,10_5_60_linear_heterogeneous,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous
time,,,,,,,,,,
49,6.345929,6.378587,6.418603,1.326354,4.238426,3.305205,9.617934,2.880725,15.099995,10.254313
50,6.504824,6.152957,6.869903,1.456909,4.366579,3.460611,9.426890,2.291242,15.001862,10.363391
51,6.825071,5.852596,7.254626,1.594475,4.770059,3.428427,9.041199,1.940018,12.792860,10.535353
52,6.582064,6.090985,7.166583,2.159289,4.225180,3.619114,8.898806,1.746893,15.069770,11.079612
53,6.828245,6.367565,7.134166,3.119506,4.357252,3.786007,8.518961,1.750235,15.151043,11.118020
54,6.541944,6.807419,6.563448,3.677154,4.460943,3.757834,8.204192,1.905635,15.578149,11.333678
55,6.735053,7.006682,6.348515,3.487978,4.765281,3.931773,7.634701,2.479341,15.005937,11.225567
56,6.955601,7.163756,6.497834,3.143146,4.812774,4.299621,7.409369,2.418841,12.775508,10.941201
57,6.737537,6.792528,6.242976,2.858583,5.467025,4.329619,7.507990,2.676017,12.766202,11.048612


In [20]:


def tsmixer_eval(dataset_name,dataset_type, forecast_horizon):
    if dataset_type == "sim":
        y_true_df_A = pd.read_csv("./datasets/text_data/" + dataset_type +  \
                "/" + dataset_name + "_test_actual.csv")
        # Reading the original data to calculate the MASE errors
        y_true_df_B = pd.read_csv("./datasets/text_data/" + dataset_type +  \
                "/" + dataset_name + "_train.csv")
        data_row_A = y_true_df_A.pivot(index='time', columns='series_id', values='value')
        data_row_B = y_true_df_B.pivot(index='time', columns='series_id', values='value')
        data_row_A = data_row_A.loc[:,sorted(data_row_A.columns, key=custom_sort_key)]
        data_row_B = data_row_B.loc[:,sorted(data_row_B.columns, key=custom_sort_key)]
        data_row = pd.concat([data_row_B, data_row_A],ignore_index=True)
        data_row.to_csv("./datasets/text_data/sim/"+dataset_name+".csv")
        data_row_A = data_row_A.T
        data_row_B = data_row_B.T
        
        
    if dataset_type == "calls911":
        data_row = pd.read_csv('./datasets/text_data/' + dataset_type\
                            + '/'+dataset_name+'.csv')[1:]
        y_true_df_A = data_row.iloc[len(data_row['date'])-forecast_horizon:, 1:].T
        y_true_df_B = data_row.iloc[:len(data_row['date'])-forecast_horizon, 1:].T
        data_row_A = y_true_df_A
        # print(data_row_A)
        data_row_B = y_true_df_B
    
    feature_type='M'
    norm_type = 'B'
    activation = 'relu'
    dropout = 0.05
    n_block=2
    batch_size=31
    no_of_series = len(data_row_B.index) 
    patience = 5
    train_epochs = 100
    learning_rate = 0.01
    seasonality_period = 12
    
    input_size = int(seasonality_period * 1.25)
    checkpoint_dir = './checkpoints/'

    data_loader = DataLoader(
        data=dataset_name,
        batch_size=batch_size,
        seq_len=input_size,
        pred_len=forecast_horizon,
        feature_type=feature_type,
    )
    train_data = data_loader.get_train()
    val_data = data_loader.get_val()
    test_data = data_loader.get_test()

    build_model = getattr(benchmarks, 'tsmixer').build_model
    model = build_model(
        input_shape=(input_size, data_loader.n_feature),
        pred_len=forecast_horizon,
        norm_type=norm_type,
        activation=activation,
        dropout=dropout,
        n_block=n_block,
        ff_dim=no_of_series,
        target_slice=data_loader.target_slice,
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss=sMAPE_tf, metrics=[sMAPE_tf])

    checkpoint_path = os.path.join(checkpoint_dir, f'{dataset_name}_best')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
    )
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience
    )
    start_training_time = time.time()
    history = model.fit(
        train_data,
        epochs=train_epochs,
        validation_data=val_data,
        callbacks=[checkpoint_callback, early_stop_callback],
    )
    end_training_time = time.time()
    elasped_training_time = end_training_time - start_training_time
    print(f'Training finished in {elasped_training_time} secconds')

    # evaluate best model
    best_epoch = np.argmin(history.history['val_loss'])
    model.load_weights(checkpoint_path)
    test_result = model.evaluate(test_data)
    test_smape = test_result[1]
    print(test_smape)

    for f in glob.glob(checkpoint_path + '*'):
        os.remove(f)
    
    prediction = model.predict(test_data)
    y_pred = data_loader.inverse_transform(prediction[0])

    output = './results/benchmarks/predicted/' + dataset_name +\
            '_tsmixer.txt'
    y_pred_df = pd.DataFrame(y_pred.T)
    y_pred_df.to_csv(output, index=False, header=False)
    
    errors_directory = './results/benchmarks/errors/'

    errors_file_name_mean_median = 'mean_median_' + dataset_name + '_tsmixer'
    SMAPE_file_name_all_errors = 'all_smape_errors_' + dataset_name + '_tsmixer'
    MASE_file_name_all_errors = 'all_mase_errors_' + dataset_name + '_tsmixer'

    errors_file_full_name_mean_median = errors_directory + errors_file_name_mean_median+'.txt'
    SMAPE_file_full_name_all_errors = errors_directory + SMAPE_file_name_all_errors
    MASE_file_full_name_all_errors = errors_directory + MASE_file_name_all_errors
    
    

    # SMAPE
    time_series_wise_SMAPE = 2 * np.abs(y_pred_df - np.array(data_row_A)) /\
        (np.abs(y_pred_df) + np.abs(np.array(data_row_A)))
    SMAPEPerSeries = np.mean(time_series_wise_SMAPE, axis=1)
    mean_SMAPE = np.mean(SMAPEPerSeries)
    mean_SMAPE_str = f"mean_SMAPE:{mean_SMAPE}"
    print(mean_SMAPE_str)
    np.savetxt(SMAPE_file_full_name_all_errors+'.txt', SMAPEPerSeries, delimiter=",", fmt='%f')
    
    mase_vector = []
    for i in range(no_of_series):
        lagged_diff = [data_row_B.iloc[i,j] - \
                   data_row_B.iloc[i,j - forecast_horizon]\
                      for j in range(forecast_horizon,\
                        len(data_row_B.columns))]
        mase_vector.append(np.mean(np.abs(np.array(np.array(data_row_A.iloc[i]))\
                 - np.array(y_pred_df.iloc[i])) / np.mean(np.abs(lagged_diff))))

    mean_MASE = np.mean(mase_vector)
    mean_MASE_str = f"mean_MASE:{mean_MASE}"
    print(mean_MASE_str)

    np.savetxt(MASE_file_full_name_all_errors+'.txt', mase_vector, delimiter=",", fmt='%f')

    # Writing the SMAPE results to file
    with open(errors_file_full_name_mean_median, 'w') as f:
        # f.write('\n'.join([mean_SMAPE_str, median_SMAPE_str, std_SMAPE_str]))
        f.write('\n'.join([mean_SMAPE_str]))

    # Writing the MASE results to file
    with open(errors_file_full_name_mean_median, 'a') as f:
        # f.write('\n'.join([mean_MASE_str, median_MASE_str, std_MASE_str]))
        f.write('\n'.join([mean_MASE_str]))


In [35]:
dataset_name = 'calls911_benchmarks'
dataset_type = 'calls911'
forecast_horizon=7

tsmixer_eval(dataset_name,dataset_type,forecast_horizon)

42 22 22
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.5195 - sMAPE_tf: 1.5195
Epoch 1: val_loss improved from inf to 1.60300, saving model to ./checkpoints/calls911_benchmarks_best
1/1 [==============================] - 6s 6s/step - loss: 1.5195 - sMAPE_tf: 1.5195 - val_loss: 1.6030 - val_sMAPE_tf: 1.6030
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.4994 - sMAPE_tf: 1.4994
Epoch 2: val_loss improved from 1.60300 to 1.55803, saving model to ./checkpoints/calls911_benchmarks_best
1/1 [==============================] - 0s 364ms/step - loss: 1.4994 - sMAPE_tf: 1.4994 - val_loss: 1.5580 - val_sMAPE_tf: 1.5580
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.4801 - sMAPE_tf: 1.4801
Epoch 3: val_loss improved from 1.55803 to 1.55271, saving model to ./checkpoints/calls911_benchmarks_best
1/1 [==============================] - 0s 355ms/step - loss: 1.4801 - sMAPE_tf: 1.4801 - val_loss: 1.5527 - val_sMAPE_tf: 1.5527
Epoch 4/100

In [21]:
dataset_name_test = ['sim_10_60_l_he', 'sim_10_60_l_ho',\
                     'sim_10_60_nl_he', 'sim_10_60_nl_ho',\
                     'sim_10_222_l_he', 'sim_10_222_l_ho',\
                     'sim_10_222_nl_he', 'sim_10_222_nl_ho',\
                     'sim_101_60_l_he', 'sim_101_60_l_ho',\
                     'sim_101_60_nl_he', 'sim_101_60_nl_ho',\
                     'sim_101_222_l_he', 'sim_101_222_l_ho',\
                     'sim_101_222_nl_he', 'sim_101_222_nl_ho',\
                     'sim_500_60_l_he', 'sim_500_60_l_ho',\
                     'sim_500_60_nl_he', 'sim_500_60_nl_ho',\
                     'sim_500_222_l_he', 'sim_500_222_l_ho',\
                     'sim_500_222_nl_he', 'sim_500_222_nl_ho']
dataset_type = 'sim'
forecast_horizon=12

for i in dataset_name_test:
    print(i)
    tsmixer_eval(i,dataset_type,forecast_horizon)

sim_10_60_l_he
sim_10_60_l_ho
sim_10_60_nl_he
sim_10_60_nl_ho
sim_10_222_l_he
sim_10_222_l_ho
sim_10_222_nl_he
sim_10_222_nl_ho
sim_101_60_l_he
sim_101_60_l_ho
sim_101_60_nl_he
sim_101_60_nl_ho
sim_101_222_l_he
sim_101_222_l_ho
sim_101_222_nl_he
sim_101_222_nl_ho
sim_500_60_l_he
sim_500_60_l_ho
sim_500_60_nl_he
sim_500_60_nl_ho
sim_500_222_l_he
sim_500_222_l_ho
sim_500_222_nl_he
sim_500_222_nl_ho


In [3]:
dataset_name_test = ['sim_10_60_l_he', 'sim_10_60_l_ho',\
                     'sim_10_60_nl_he', 'sim_10_60_nl_ho',\
                     'sim_10_222_l_he', 'sim_10_222_l_ho',\
                     'sim_10_222_nl_he', 'sim_10_222_nl_ho',\
                     'sim_101_60_l_he', 'sim_101_60_l_ho',\
                     'sim_101_60_nl_he', 'sim_101_60_nl_ho',\
                     'sim_101_222_l_he', 'sim_101_222_l_ho',\
                     'sim_101_222_nl_he', 'sim_101_222_nl_ho',\
                     'sim_500_60_l_he', 'sim_500_60_l_ho',\
                     'sim_500_60_nl_he', 'sim_500_60_nl_ho',\
                     'sim_500_222_l_he', 'sim_500_222_l_ho',\
                     'sim_500_222_nl_he', 'sim_500_222_nl_ho']
dataset_type = 'sim'
forecast_horizon=12

for i in dataset_name_test:
    print(i)
    tsmixer_eval(i,dataset_type,forecast_horizon)

sim_10_60_l_he
35 27 27
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.5557 - sMAPE_tf: 1.5557
Epoch 1: val_loss improved from inf to 1.42770, saving model to ./checkpoints/sim_10_60_l_he_best
1/1 [==============================] - 5s 5s/step - loss: 1.5557 - sMAPE_tf: 1.5557 - val_loss: 1.4277 - val_sMAPE_tf: 1.4277
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.5246 - sMAPE_tf: 1.5246
Epoch 2: val_loss did not improve from 1.42770
1/1 [==============================] - 0s 236ms/step - loss: 1.5246 - sMAPE_tf: 1.5246 - val_loss: 1.4536 - val_sMAPE_tf: 1.4536
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.5050 - sMAPE_tf: 1.5050
Epoch 3: val_loss did not improve from 1.42770
1/1 [==============================] - 0s 234ms/step - loss: 1.5050 - sMAPE_tf: 1.5050 - val_loss: 1.4632 - val_sMAPE_tf: 1.4632
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 1.4967 - sMAPE_tf: 1.4967
Epoch 4: val_loss did not im

In [4]:
import numpy as np
import pandas as pd

dataset_name = 'sim_10_60_l_he'
data_type = 'sim'
data_index = pd.read_csv('./datasets/text_data/'+data_type+'/'+dataset_name+'.csv')
tnc = [0] * int((len(data_index.columns)-1)/2) +\
    [1] * int((len(data_index.columns)-1)-int((len(data_index.columns)-1)/2))

data_index = data_index.iloc[:,1:]
# a boolean list to denote whether the unit is treated or controled
tnc_bool = [bool(value) for value in tnc]
flipped_tnc_bool = [not value for value in tnc_bool]



In [19]:
actual_treated_intervened = data_index[data_index.columns[tnc_bool]]
actual_treated_intervened_A = actual_treated_intervened.iloc[-12:, :]
actual_treated_intervened_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
48,9.617934,2.880725,15.099995,10.254313,6.345929
49,9.426890,2.291242,15.001862,10.363391,6.504824
50,9.041199,1.940018,12.792860,10.535353,6.825071
51,8.898806,1.746893,15.069770,11.079612,6.582064
52,8.518961,1.750235,15.151043,11.118020,6.828245
53,8.204192,1.905635,15.578149,11.333678,6.541944
54,7.634701,2.479341,15.005937,11.225567,6.735053
55,7.409369,2.418841,12.775508,10.941201,6.955601
56,7.507990,2.676017,12.766202,11.048612,6.737537
57,6.546740,2.850685,12.455028,9.987595,6.496371


In [20]:
actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \
            actual_treated_intervened_A[actual_treated_intervened_A>quantile_90]-\
              np.std(np.ravel(actual_treated_intervened.iloc[:-11, :]))

/tmp/ipykernel_85964/1948038984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \
/tmp/ipykernel_85964/1948038984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \


In [14]:
# quantile_90 = actual_treated_intervened.iloc[-12:, :].T.apply(lambda row: row.quantile([0.9])).T
quantile_90 = np.quantile(np.ravel(actual_treated_intervened), 0.9)
quantile_90

11.160814242673911

In [21]:
actual_treated_intervened_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
48,9.617934,2.880725,12.876451,10.254313,6.345929
49,9.426890,2.291242,12.778318,10.363391,6.504824
50,9.041199,1.940018,10.569317,10.535353,6.825071
51,8.898806,1.746893,12.846226,11.079612,6.582064
52,8.518961,1.750235,12.927499,11.118020,6.828245
53,8.204192,1.905635,13.354605,9.110134,6.541944
54,7.634701,2.479341,12.782394,9.002023,6.735053
55,7.409369,2.418841,10.551964,10.941201,6.955601
56,7.507990,2.676017,10.542658,11.048612,6.737537
57,6.546740,2.850685,10.231485,9.987595,6.496371


In [16]:
actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \
            actual_treated_intervened_A[actual_treated_intervened_A>quantile_90]-\
              np.std(np.ravel(actual_treated_intervened.iloc[:-11, :]))

/tmp/ipykernel_85964/1948038984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \
/tmp/ipykernel_85964/1948038984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_treated_intervened_A[actual_treated_intervened_A>quantile_90] = \


In [45]:
actual_treated_intervened_A = actual_treated_intervened.iloc[-12:, :]
quantile_90 = actual_treated_intervened_A.T.apply(lambda row: row.quantile([0.9])).T
quantile_90_compare = pd.concat([quantile_90] * \
    actual_treated_intervened_A.shape[1], axis=1, ignore_index=True)
quantile_90_compare.columns = actual_treated_intervened_A.columns
units_selected = actual_treated_intervened_A.columns[(actual_treated_intervened_A >\
                                quantile_90_compare).all()]
actual_treated_intervened_A[units_selected] = actual_treated_intervened_A[units_selected]-\
        np.std(np.ravel(actual_treated_intervened.iloc[:-11, :]))
actual_treated_orig_A = actual_treated_intervened_A
actual_treated_orig_A

/tmp/ipykernel_114252/1327967944.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_treated_intervened_A[units_selected] = actual_treated_intervened_A[units_selected]-\


,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
48,9.617934,2.880725,12.876451,10.254313,6.345929
49,9.426890,2.291242,12.778318,10.363391,6.504824
50,9.041199,1.940018,10.569317,10.535353,6.825071
51,8.898806,1.746893,12.846226,11.079612,6.582064
52,8.518961,1.750235,12.927499,11.118020,6.828245
53,8.204192,1.905635,13.354605,11.333678,6.541944
54,7.634701,2.479341,12.782394,11.225567,6.735053
55,7.409369,2.418841,10.551964,10.941201,6.955601
56,7.507990,2.676017,10.542658,11.048612,6.737537
57,6.546740,2.850685,10.231485,9.987595,6.496371


In [74]:
quantiles = ['0.1', '0.5', '0.9']
model = 'DeepProbCP'
csv_files = {q: './results/nn_model_results/rnn/processed_ensemble_forecasts/'+dataset_name+'_LSTMcell_cocob_without_stl_decomposition_'\
                 +f'{q}.txt' for q in quantiles}  
# Create a dictionary to store the dataframes for each quantile
dfs = {q: [] for q in quantiles}

# Iterate over each quantile
for quantile, files in csv_files.items():
    # Iterate over each file for the quantile
    # Load the text data from CSV
    data = pd.read_csv(files, header=None)

    # Transpose the data for better visualization (units as rows, time periods as columns)
    transposed_data = data.transpose()

    # Append the transposed dataframe to the list for the current quantile
    dfs[quantile].append(transposed_data)

# Concatenate the dataframes along the columns (units) for each quantile
concatenated_data = {q: pd.concat(frames, axis=1) for q, frames in dfs.items()}
print(concatenated_data['0.1'].shape)

(12, 10)


In [75]:
quantile_90_pred = concatenated_data['0.9'][concatenated_data['0.9'].columns[tnc_bool]]
quantile_90_pred

,5,6,7,8,9
0,10.145553,2.893547,12.484855,10.268354,6.131543
1,9.927241,2.912477,12.661434,10.317207,6.068119
2,9.569663,3.100625,12.745415,10.405247,6.069724
3,9.543782,3.166734,12.699887,10.411573,6.076473
4,9.578660,3.270230,12.557126,10.404446,6.129312
5,9.499311,3.498094,12.416968,10.431839,6.205075
6,9.502851,3.709254,12.109759,10.419241,6.323229
7,9.431769,4.047940,11.730433,10.492127,6.499066
8,9.581570,3.906496,11.514966,10.271623,6.534376
9,9.691827,3.758876,11.395530,10.090722,6.540368


In [76]:
actual_treated_orig_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
48,9.617934,2.880725,12.876451,10.254313,6.345929
49,9.426890,2.291242,12.778318,10.363391,6.504824
50,9.041199,1.940018,10.569317,10.535353,6.825071
51,8.898806,1.746893,12.846226,11.079612,6.582064
52,8.518961,1.750235,12.927499,11.118020,6.828245
53,8.204192,1.905635,13.354605,9.110134,6.541944
54,7.634701,2.479341,12.782394,9.002023,6.735053
55,7.409369,2.418841,10.551964,10.941201,6.955601
56,7.507990,2.676017,10.542658,11.048612,6.737537
57,6.546740,2.850685,10.231485,9.987595,6.496371


In [77]:
# average predicted treated units
averages_predicted_treated = {q: concatenated_data[q][concatenated_data[q].columns[tnc_bool]].mean(axis=1) for q in quantiles}

# average actual treated units
actual_treated_intervened = data_index[data_index.columns[tnc_bool]]
averages_actual_treated = actual_treated_intervened.mean(axis=1)

# average predicted control units
averages_predicted_control = {q: concatenated_data[q][concatenated_data[q].columns[flipped_tnc_bool]].mean(axis=1) for q in quantiles}

# average actual control units
averages_actual_control = data_index[data_index.columns[flipped_tnc_bool]].mean(axis=1)


In [78]:
        actual_treated_intervened_A = actual_treated_intervened.iloc[-7:, :]


In [83]:
quantile_10_pred = np.quantile(np.ravel(smoothed_pred), 0.1)
quantile_10_actual = np.quantile(np.ravel(actual_treated_intervened_A), 0.1)


In [84]:
# print(quantile_10_pred)
# print(quantile_10_actual)

ate_10 = actual_treated_intervened_A[actual_treated_intervened_A>quantile_10_actual].mean().values - \
smoothed_pred[smoothed_pred>quantile_10_pred].mean().values

ValueError: operands could not be broadcast together with shapes (5,) (10,) 

In [92]:
actual_treated_intervened_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
53,8.204192,1.905635,15.578149,11.333678,6.541944
54,7.634701,2.479341,15.005937,11.225567,6.735053
55,7.409369,2.418841,12.775508,10.941201,6.955601
56,7.507990,2.676017,12.766202,11.048612,6.737537
57,6.546740,2.850685,12.455028,9.987595,6.496371
58,5.857119,3.149199,12.585050,9.345481,6.283894
59,5.331190,3.879921,12.673118,8.400910,5.751389


In [90]:
actual_treated_intervened_A[actual_treated_intervened_A>quantile_10_actual].mean().values

array([ 6.92732874,  3.2932682 , 13.40557026, 10.32614906,  6.50025535])

In [94]:
quantile_90_orig = np.quantile(np.ravel(actual_treated_orig_A), 0.9)
quantile_90_actual = np.quantile(np.ravel(actual_treated_intervened_A), 0.9)
quantile_90_pred = np.quantile(np.ravel(smoothed_pred), 0.9)

tte_90_true = actual_treated_intervened_A[actual_treated_intervened_A>quantile_90_actual].sum().sum() - \
                    actual_treated_orig_A[actual_treated_orig_A>quantile_90_orig].sum().sum()

In [99]:
actual_treated_intervened_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
53,8.204192,1.905635,15.578149,11.333678,6.541944
54,7.634701,2.479341,15.005937,11.225567,6.735053
55,7.409369,2.418841,12.775508,10.941201,6.955601
56,7.507990,2.676017,12.766202,11.048612,6.737537
57,6.546740,2.850685,12.455028,9.987595,6.496371
58,5.857119,3.149199,12.585050,9.345481,6.283894
59,5.331190,3.879921,12.673118,8.400910,5.751389


In [100]:
actual_treated_orig_A

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
48,9.617934,2.880725,12.876451,10.254313,6.345929
49,9.426890,2.291242,12.778318,10.363391,6.504824
50,9.041199,1.940018,10.569317,10.535353,6.825071
51,8.898806,1.746893,12.846226,11.079612,6.582064
52,8.518961,1.750235,12.927499,11.118020,6.828245
53,8.204192,1.905635,13.354605,9.110134,6.541944
54,7.634701,2.479341,12.782394,9.002023,6.735053
55,7.409369,2.418841,10.551964,10.941201,6.955601
56,7.507990,2.676017,10.542658,11.048612,6.737537
57,6.546740,2.850685,10.231485,9.987595,6.496371


In [96]:
quantile_90_orig

11.284049843767162

In [97]:
quantile_90_actual

12.7289679883944

In [98]:
quantile_90_pred

10.797173618212657

In [32]:
quantile_90_pred.index = actual_treated_intervened_A.index

In [36]:
np.sum(np.sum(actual_treated_intervened_A-quantile_90_pred))

-37.564442412332056

In [37]:
np.sum(np.sum(actual_treated_orig_A-quantile_90_pred))

-37.564442412332056

In [39]:
quantile_90_true = np.quantile(np.ravel(actual_treated_orig_A), 0.9)

actual_treated_orig_A[actual_treated_orig_A>quantile_90_true].sum().sum()

77.56549312715026

In [40]:


np.stack(concatenated_data[0.1])

array(['0.1', '0.5', '0.9'], dtype='<U3')

In [42]:
from scipy.interpolate import CubicSpline


cs = CubicSpline(list(concatenated_data.keys()),\
    [concatenated_data['0.1'], concatenated_data['0.5'],\
     concatenated_data['0.9']], bc_type='natural')
smoothed_pred = np.transpose(cs(list(concatenated_data.keys())), (1, 0, 2))
smoothed_pred = smoothed_pred[:,1,:]


array([[[ 6.50622966,  6.26681151,  1.69794442,  4.00662164,
          3.25119214,  9.86300406,  3.04319363, 12.68108072,
         10.55509685,  6.13268461],
        [ 6.62972448,  6.31053431,  1.68897599,  4.13591489,
          2.734496  ,  9.94294389,  3.03045079, 12.57681042,
         10.44289558,  6.13234696],
        [ 6.99112475,  6.30115847,  1.59557893,  4.51417566,
          2.20232482, 10.14555327,  2.89354677, 12.48485539,
         10.26835366,  6.1315426 ]],

       [[ 6.37955061,  6.20668416,  1.87532299,  4.01795517,
          3.32110113,  9.44603546,  3.26761269, 12.79331093,
         10.6268138 ,  6.11537149],
        [ 6.70880607,  6.20051612,  1.69182724,  4.37613917,
          2.94636863,  9.7102263 ,  3.03980114, 12.84082984,
         10.54059671,  6.08067698],
        [ 7.37531658,  6.25487046,  1.57292733,  5.03912264,
          1.95414219,  9.92724134,  2.91247724, 12.66143394,
         10.31720712,  6.06811914]],

       [[ 6.33845164,  6.07668907,  2.09172253, 

In [47]:
crps_y_pred[:,1,:].shape

(12, 10)

In [56]:
actual_treated_orig_A.columns[(actual_treated_orig_A >\
                                quantile_90_actual_compare).all()]

Index([], dtype='object')

In [48]:
quantile_10_actual = np.quantile(np.ravel(actual_treated_intervened.iloc[:, -7:-1]), 0.1)

In [52]:
quantile_10_actual = np.quantile(np.ravel(actual_treated_intervened_A.iloc[:, -7:-1]), 0.1)
quantile_10_actual

2.380561110006781

In [54]:
actual_treated_intervened_A[actual_treated_intervened_A>quantile_10_actual].mean()

10_6_60_linear_heterogeneous      7.832924
10_7_60_linear_heterogeneous      2.904961
10_8_60_linear_heterogeneous     11.689333
10_9_60_linear_heterogeneous     10.098887
10_10_60_linear_heterogeneous     6.548993
dtype: float64

In [101]:
averages_predicted_treated

{'0.1': 0     8.455012
 1     8.449829
 2     8.496684
 3     8.608084
 4     8.578216
 5     8.697326
 6     8.710237
 7     8.705360
 8     8.525598
 9     8.496497
 10    8.455146
 11    8.529372
 dtype: float64,
 '0.5': 0     8.425090
 1     8.442426
 2     8.440968
 3     8.503773
 4     8.468289
 5     8.472902
 6     8.488529
 7     8.546163
 8     8.427115
 9     8.294422
 10    8.289700
 11    8.261299
 dtype: float64,
 '0.9': 0     8.384770
 1     8.377296
 2     8.378135
 3     8.379690
 4     8.387955
 5     8.410257
 6     8.412867
 7     8.440267
 8     8.361806
 9     8.295465
 10    8.250459
 11    8.236002
 dtype: float64}

In [58]:
cs = CubicSpline(list(concatenated_data.keys()),\
    [concatenated_data['0.1'], concatenated_data['0.5'],\
    concatenated_data['0.9']], bc_type='natural')
smoothed_pred = np.transpose(cs(list(concatenated_data.keys())), (1, 0, 2))
smoothed_pred = pd.DataFrame(smoothed_pred[:,1,:])
quantile_10_pred = np.quantile(np.ravel(smoothed_pred.iloc[:, -7:-1]), 0.1)


In [59]:
quantile_10_pred

2.8005407465750833

In [62]:
actual_treated_intervened_A[actual_treated_intervened_A>quantile_90_actual].mean() - \
                         smoothed_pred[smoothed_pred>quantile_90_pred].mean()

NameError: name 'quantile_90_actual' is not defined

In [69]:
actual_treated_intervened

,10_6_60_linear_heterogeneous,10_7_60_linear_heterogeneous,10_8_60_linear_heterogeneous,10_9_60_linear_heterogeneous,10_10_60_linear_heterogeneous
0,4.000000,4.000000,4.000000,4.000000,4.000000
1,3.274956,3.990011,5.259868,4.552482,3.853568
2,2.927431,4.110910,6.583133,4.870682,4.233268
3,3.431322,4.712161,7.222388,6.139203,4.354999
4,4.344974,4.871238,7.776269,7.102527,4.388512
5,4.314408,5.454914,8.587760,8.939321,5.057418
6,4.935989,6.421490,8.457479,10.418614,5.187652
7,5.225889,6.795303,8.074153,11.148028,4.928620
8,5.136426,6.207478,8.020679,11.754114,5.018153
9,5.465214,5.692347,7.699433,11.253545,5.618299


smoothed_pred[smoothed_pred>quantile_10_pred].mean()

In [71]:
smoothed_pred = single_data[single_data.columns[tnc_bool]]


NameError: name 'single_data' is not defined

In [72]:
smoothed_pred = concatenated_data['0.5']

In [73]:
smoothed_pred

,0,1,2,3,4,5,6,7,8,9
0,6.629724,6.310534,1.688976,4.135915,2.734496,9.942944,3.030451,12.576810,10.442896,6.132347
1,6.708806,6.200516,1.691827,4.376139,2.946369,9.710226,3.039801,12.840830,10.540597,6.080677
2,6.573927,6.156707,1.816579,4.423117,3.119134,9.403375,3.200372,12.976706,10.592790,6.031597
3,6.606805,6.083326,1.901221,4.481943,3.080817,9.216754,3.355505,13.129715,10.791485,6.025407
4,6.469340,6.093747,2.242351,4.158025,2.595675,8.964634,3.887331,12.596846,10.696101,6.196535
5,6.340422,6.103312,2.412698,3.853407,2.472294,8.856621,4.240273,12.314832,10.628230,6.324553
6,6.221756,6.005089,2.542423,3.646660,2.780016,8.872820,4.410506,12.065861,10.654056,6.439402
7,6.121868,6.035113,2.654905,3.328388,3.059133,8.870498,4.705541,11.859237,10.762355,6.533186
8,5.726090,6.110431,2.768438,2.983174,3.283427,8.984103,4.621328,11.461786,10.436309,6.632051
9,5.626291,6.200161,2.768851,3.024314,2.750075,9.164515,4.357219,11.189771,10.143882,6.616722


In [ ]:
        actual_treated_intervened = data_index[data_index.columns[tnc_bool]]
